In [38]:
import cv2
import numpy as np
from PIL import Image
import os
import warnings


def colorTransfer(src,trgt): #Since I will use the color transfer part in part 2, it is defined as a function.
    
    #Source Image Part
    source = cv2.cvtColor(src, cv2.COLOR_BGR2LAB).astype("float32")#I converted rbg to lab to split "l" "a" and "b"
    (lS, aS, bS) = cv2.split(source)#I splitted into LAB parts
    #I calculated means and standart deviations of Source LAB 
    (L_MSource, L_SSource) =(lS.mean(), lS.std())
    (A_MSource, A_SSource) =(aS.mean(), aS.std())
    (B_MSource, B_SSource) =(bS.mean(), bS.std())

    #Source Image Part
    target = cv2.cvtColor(trgt, cv2.COLOR_BGR2LAB).astype("float32")#I converted rbg to lab to split "l" "a" and "b"
    (lT, aT, bT) = cv2.split(target) #I splitted into LAB parts
    #I calculated means and standart deviations of Target LAB 
    (L_MTarget, L_STarget) =(lT.mean(), lT.std())
    (A_MTarget, A_STarget) =(aT.mean(), aT.std())
    (B_MTarget, B_STarget) =(bT.mean(), bT.std())

    #I applied the extraction process found in the third step of Part 1
    (l, a, b) = cv2.split(source)
    l -=L_MSource
    a -=A_MSource
    b -=B_MSource

    #I've handled the case where source is 0 in the Target/Source operation.
    if(L_SSource==0):
        L_SSource+=0.0001
    if(A_SSource==0):
        A_SSource+=0.0001
    if(B_SSource==0):
        B_SSource+=0.0001
    
    #I applied the extraction process found in the fourth step of Part 1
    l =(L_STarget/L_SSource) * l
    a =(A_STarget/A_SSource) * a
    b =(B_STarget/B_SSource) * b

    #I applied the extraction process found in the fifth step of Part 1
    l +=L_MTarget
    a +=A_MTarget
    b +=B_MTarget

    #I used the clip function to limit
    l =np.clip(l, 0, 255)
    a =np.clip(a, 0, 255)
    b =np.clip(b, 0, 255)

    result =cv2.merge([l, a, b])#I combined LAB into one element 
    result =cv2.cvtColor(result.astype("uint8"), cv2.COLOR_LAB2BGR) #I converted lab to rgb
    return result

def ncc(var1,var2): #I used the ncc algorithm that I used Assignment 1 
    
    var1=var1-var1.mean(axis=0)
    var2=var2-var2.mean(axis=0)
    return np.sum(((var1/np.linalg.norm(var1))*(var2/np.linalg.norm(var2))))

warnings.filterwarnings("ignore")
photos = ['01','02','03','05','06','08','09','11','15','20','24','29','50','51','56'] #I defined output names array

#Part 1 Operations
if not os.path.exists('Result-Part1'):
    os.makedirs('Result-Part1')
    
for i in range(len(photos)):
    im = cv2.imread('data-2/in_'+photos[i]+'.png')
    im2 = cv2.imread('data-2/tar_'+photos[i]+'.png')
    
    result=colorTransfer(im,im2)
    cv2.imwrite('Result-Part1/res_'+photos[i]+'.png',result)
        
        
#Part 2 Operations
if not os.path.exists('Result-Part2'):
    os.makedirs('Result-Part2')
    
for m in range(len(photos)):                         
    img1 = cv2.imread('data-2/in_'+photos[m]+'.png')
    img2 = cv2.imread('data-2/tar_'+photos[m]+'.png')

    if(img1.shape[0]>img2.shape[0]): 
        img1=img1[0:img2.shape[0],:,:]
    else:
        img2=img2[0:img1.shape[0],:,:]
    
    if(img1.shape[1]>img2.shape[1]):
        img1 = img1[:,0:img2.shape[1],:]
    else:
        img2 = img2[:,0:img1.shape[1],:]

    M = img1.shape[0]//5 #I splitted photos into 25 part
    N = img1.shape[1]//5

    temp= -1
    for k in range(0,img1.shape[0],M): #I used these for loops to loop through the photos piece by piece
        for l in range(0,img1.shape[1],N):
            temp=-1
            for i in range(0,img2.shape[0],M):
                for j in range(0,img2.shape[1],N):
                    if(img1[k:k+M,l:l+N].shape != img2[i:i+M,j:j+N].shape):
                        continue
                    a = ncc(img1[k:k+M,l:l+N],img2[i:i+M,j:j+N]) 
                    if(a>temp):
                        temp= a
                        temp1=i
                        temp2=j
            img1[k:k+M,l:l+N] = colorTransfer(img1[k:k+M,l:l+N],img2[temp1:temp1+M,temp2:temp2+N])

    cv2.imwrite('Result-Part2/res_'+photos[m]+'.png',img1)    
    
